In [9]:
# General Stuff:
import numpy as np
import pandas as pd

# Our Stuff:
from models.AutoRecBase import AutoRecBase
from models.VarAutoRec import VarAutoRec
from models.MF import MF


from scripts.get_data import download_2_data_sets, ratings_to_train_test, ratings_to_train_test_u
from scripts.get_2_other_data import get_2_other_datasets, secondary_to_train_test, secondary_to_train_test_u
from utils.evaluate import evaluate_model
from utils.loading_utils import load_model, save_model

import torch
from torch import nn
import pytorch_lightning as pl

# Visualization Stuff
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 240)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1660 Ti


# AutoRec - AutoEncoders Meet Collaborative Filtering - PyTorch

<a id="toc"></a>
## Table of Content
1. [Introduction](#introduction)
1. [Conclusions](#conclusions)

<a id="introduction"></a>

## Introduction

In this notebook we will review a collaborative filtering approach using autoencoders, as suggested by Sedhain et al, in their 2015 paper "AutoRec: Autoencoders Meet Collaborative Filtering".

We will then introduce several improvements and asses them.

[Table of content](#toc)

In [10]:
is_default_dataset = False
if is_default_dataset:
    download_2_data_sets()
else:
    get_2_other_datasets()

In [12]:
if is_default_dataset:
    train_loader, val_loader = ratings_to_train_test(1,0, 1,10)
    mf_train_loader, mf_val_loader =  ratings_to_train_test_u(dataset_size=1,
                                                              validation_partition=0,
                                                              train_partition=1,
                                                              batch_size=10)
else:
    train_loader, val_loader = secondary_to_train_test('flixster',0, 1,10)
    mf_train_loader, mf_val_loader =  secondary_to_train_test_u('flixster',
                                                                validation_partition=0,
                                                                train_partition=1,
                                                                batch_size=10)


In [13]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...

Go to:  [TensorBoard](http://localhost:6006)

In [14]:
models_dict = {} # (model,ephoc,lr): loss
models_state = {} # (model,ephoc,lr): model.state_dict()
models = [
    AutoRecBase,
    VarAutoRec,
    MF
]
lrs = [0.001,0.005,0.01]
activations = [nn.PReLU, nn.Sigmoid]

Sanity check:

In [15]:
for x, y, m in val_loader:
    print(x[:,0])
    break
for x, y, m in mf_val_loader:
    print(x[0])
    break
for x, y, r in mf_train_loader:
    print(x)
    print(y)
    print(r)
    break
# model(x)[:,0]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])
tensor(7)
tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
tensor([ 929, 1000, 1135, 1165, 1183, 1187, 1291, 1561, 1576, 1577])
tensor([5.0000, 5.0000, 4.5000, 5.0000, 3.5000, 4.5000, 4.5000, 4.5000, 4.0000,
        4.0000], dtype=torch.float64)


In [16]:
print(f"Number of users: {len(val_loader.dataset)}")
print(f"Number of items: {len(val_loader.dataset[0][1])}")

Number of users: 402
Number of items: 2876


Datasets:

In [17]:
number_of_users = len(val_loader.dataset)
number_of_items = len(val_loader.dataset[0][1])

From the original paper:

In [18]:
latent_dims = [10, 80, 300]
lambdas = [0.01, 0.1, 1, 100]

In [19]:
should_train = True
model_paths = []
if should_train:
    for model_class in models:
        for activation in activations:
            for lr in lrs:
                for latent in latent_dims:
                    for λ in lambdas:
                        model = model_class(number_of_items=number_of_items,
                                            num_of_users=number_of_users,
                                            hidden_size=latent,
                                            activation_function_1=activation,
                                            activation_function_2=activation,
                                            loss=nn.MSELoss(reduction='none'),
                                            λ=λ,
                                            lr=lr)
                        # training
                        trainer = pl.Trainer(gpus=0, max_epochs=10)
                        if type(model).__name__ == "MF":
                            trainer.fit(model,mf_train_loader, mf_val_loader)
                        else:
                            trainer.fit(model,train_loader, val_loader)

                        model_path = save_model(model_class=model_class,
                                   trainer=trainer,
                                   activation=activation,
                                   hidden_size=latent,
                                   lr=lr,
                                   λ=λ,
                                   is_default_dataset=is_default_dataset)
                        model_paths.append(model_path)
        #                 break
        #             break
        #         break
        #     break
        # break
print(model_paths)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/ido/anaconda3/envs/rs/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 28.8 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 31.6 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
60.4 K    Trainable params
0         Non-trainable params
60.4 K    Total params
0.242     Total estimated model params size (MB)


0.01
                                                              

/home/ido/anaconda3/envs/rs/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/ido/anaconda3/envs/rs/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Epoch 0:  50%|█████     | 41/82 [00:00<00:00, 118.55it/s, loss=11.4, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 1:  50%|█████     | 41/82 [00:00<00:00, 121.47it/s, loss=9.41, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 41/82 [00:00<00:00, 132.87it/s, loss=7.71, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 41/82 [00:00<00:00, 141.02it/s, loss=5.67, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 41/82 [00:00<00:00, 131.32it/s, loss=4.55, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 5:  50%|█████     | 41/82 [00:00<00:00, 140.20it/s, loss=3.64, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 6:  51%|█████     | 42/82 [00:00<00:00, 124.95it/s, loss=2.86, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 7:  51%|█████     | 42/82 [00:00<00:00, 119.97it/s, loss=2.33, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 8:  51%|█████     | 42/82 [00:00<00:00, 129.43it/s, loss=1.89, v_num=23]
Validating: 0it [00:00, ?it/s]
Epoch 9:  

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


...

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


0.01



  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 28.8 K
1 | act_1     | Sigmoid | 0     
2 | decoder   | Linear  | 31.6 K
3 | act_2     | Sigmoid | 0     
4 | loss_func | MSELoss | 0     
--------------------------------------
60.4 K    Trainable params
0         Non-trainable params
60.4 K    Total params
0.242     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/82 [00:00<?, ?it/s]               

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [10, 2876]], which is output 0 of SigmoidBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
models_eval_dict = {}
Ks = [5, 10]
i = 0

for model_class in models:
    for activation in activations:
        for lr in lrs:
            for latent in latent_dims:
                for λ in lambdas:
                    try:
                        model = load_model(model_class=model_class,
                                           activation=activation,
                                           hidden_size=latent,
                                           lr=lr,
                                           λ=λ,
                                           is_default_dataset=is_default_dataset)
                        for K in Ks:
                            (hits, ndcgs, mrrs) = evaluate_model(model, test_loader=val_loader, K=K)
                            models_eval_dict[f"row_{i}"] = [type(model).__name__, activation, latent, λ, lr, K, "HR", np.mean(hits)]
                            models_eval_dict[f"row_{i+1}"] = [type(model).__name__, activation, latent, λ, lr, K, "NDCG",np.mean(ndcgs)]
                            models_eval_dict[f"row_{i+2}"] = [type(model).__name__, activation, latent, λ, lr, K, "MRR",np.mean(mrrs)]
                            i += 3
                    except Exception as e:
                        print(e)
    #                     break
    #                 break
    #             break
    #         break
    #     break
    # break

In [ ]:
columns = ["model", "activation", "latent_dim", "lambda", "lr","topk","metric","score"]
eval_df = pd.DataFrame.from_dict(models_eval_dict, orient='index', columns=columns)
eval_df.to_csv("obj/eval_df", sep='\t')

eval_df